In [1]:
import pandas as pd
import json
import requests

Elementi da escludere perchè facenti parte di combo, sfuggiti ad altri controlli e quindi presi manualmente

In [2]:
escludere=[30276, 30303, 30304, 30853, 30907, 30908, 30914, 30915,30917,30918,30919,30920,30928,30929,30930]

Da json di wendys a json per me

In [3]:
with open("getSiteMenu.json","r") as filemenuog:
    menuog=json.load(filemenuog)

In [4]:
menuItems=menuog["menuLists"]["menuItems"]

In [5]:
salesItems=menuog["menuLists"]["salesItems"]
modifiers=menuog["menuLists"]["modifiers"]

In [6]:
menu=[]

Solo quello che mi serve cercando di evitare le combo

In [7]:
for i in range (0, len(menuItems)):
    try:
        if "comboConfigId" not in menuItems[i].keys() and menuItems[i]["menuItemId"] not in escludere:
            item={"ID": menuItems[i]["menuItemId"],"nImmagine":menuItems[i]["baseImageName"],\
           "descrizione": menuItems[i]["description"], "nome": menuItems[i]["displayName"],\
           "calorie":menuItems[i]["calorieRange"],\
            "prezzo":menuItems[i]["priceRange"], "fk":menuItems[i]["defaultItemId"]}
            menu.append(item)
    except KeyError: pass


Salvo ingredienti e categoria che erano nell'altro json

In [8]:
for i in range (0, len(menu)):
    try:
        IDsale=menu[i]["fk"]
        item=[itemS for itemS in salesItems if itemS["salesItemId"]==IDsale][0]
        menu[i]["categoria"]=item["categoryName"]
        menu[i]["ingredienti"]=[ingrediente["modifierId"] for ingrediente in item["defaultOptions"]]
        del(menu[i]["fk"])
    except KeyError: pass


In [9]:
for i in range (0, len(menu)):
    try:
        if menu[i]["categoria"]=="Cheeseburgers":
            menu[i]["tempoPreparazione"]=3.5
        elif menu[i]["categoria"]=="Chicken, Wraps & More":
            menu[i]["tempoPreparazione"]=3
        elif menu[i]["categoria"]=="Fresh-Made Salads":
            menu[i]["tempoPreparazione"]=6
        elif menu[i]["categoria"]=="Fries & Sides":
            menu[i]["tempoPreparazione"]=2
        elif menu[i]["categoria"]=="Beverages":
            menu[i]["tempoPreparazione"]=0
        elif menu[i]["categoria"]=="Frosty":
            menu[i]["tempoPreparazione"]=2
        elif menu[i]["categoria"]=="Bakery":
            menu[i]["tempoPreparazione"]=1.5
        elif menu[i]["categoria"]=="Sides & More":
            menu[i]["tempoPreparazione"]=1
        elif menu[i]["categoria"]=="Classics":
            menu[i]["tempoPreparazione"]=4
        elif menu[i]["categoria"]=="Croissants":
            menu[i]["tempoPreparazione"]=3.5
        elif menu[i]["categoria"]=="Biscuits":
            menu[i]["tempoPreparazione"]=3
    except KeyError: pass


In [11]:
ingredienti=[]
for i in range (0, len(modifiers)):
    try:
        item={"ID": modifiers[i]["modifierId"],"nome":modifiers[i]["displayName"],\
       "descrizione": modifiers[i]["description"]}
        ingredienti.append(item)
    except KeyError: pass

In [15]:
len(menu)

105

Salvo il json degli ingredienti

In [12]:
with open("ingredienti.json", "w") as writefile:
    json.dump(ingredienti, writefile, indent=1)

Salvo le pic

for item in menu:
    nPic=item["nImmagine"]
    idItem=item["ID"]
    Picture_request = requests.get("https://app.wendys.com/unified/assets/menu/pg-cropped/"+str(nPic)+"_medium_US_en.jpg")
    if Picture_request.status_code == 200:
        with open("picMenu/"+ str(idItem) + ".jpg", 'wb') as f:
            f.write(Picture_request.content)

In [13]:
for i in range(0, len(menu)):
    del(menu[i]["nImmagine"])

Salvo il json del menu

In [14]:
with open("menu.json", "w") as writefile:
    json.dump(menu, writefile, indent=1)